In [20]:
!pip install flask-ngrok

In [21]:
!pip install pyngrok --quiet

In [22]:
import numpy as np

def diagnostico(sintomas):
    # Febre, Tosse, Fadiga, Dificuldade em respirar, Idade, Gênero, Pressão arterial, Nível de colesterol

    sintomas = np.array(sintomas).reshape(1, -1)
    probabilidades = modelo.predict_proba(sintomas)
    classes = modelo.classes_

    top_n = 3
    top_doencas = np.argsort(probabilidades[0])[-top_n:][::-1]

    resultado = {
        "entrada": sintomas[0].tolist(),
        "top_diagnosticos": []
    }

    for idx in top_doencas:
        resultado["top_diagnosticos"].append({
            "doenca": classes[idx],
            "probabilidade": float(round(probabilidades[0][idx], 4))  # ou em porcentagem: round(... * 100, 2)
        })

    return resultado


In [23]:
def input_to_list(json):
    sintomas_name = ['febre', 'tosse', 'fadiga', 'dificuldade_em_respirar', 'idade', 'genero', 'pressao_arterial', 'nivel_de_colesterol']
    resultado = []
    for item in sintomas_name:
        resultado.append(json[item])
    return resultado


In [24]:
# sintomas = [1, 0, 1, 1, 19, 0, 0, 1]
# sintomas = input_to_list({'febre': 1, 'tosse': 0, 'fadiga': 1, 'dificuldade_em_respirar': 1, 'idade': 19, 'genero': 0, 'pressao_arterial': 0, 'nivel_de_colesterol': 1})
# diagnostico(sintomas)


NameError: name 'modelo' is not defined

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok, conf

import joblib
import pandas as pd
import logging

with open('/content/drive/MyDrive/data_bases/token.txt', 'r', encoding='utf-8') as f:
    conteudo = f.read()

ngrok.set_auth_token(conteudo)


conf.get_default().log_event_callback = lambda log: None
# Silenciar logs do Flask (werkzeug) e erros do Colab
logging.getLogger('werkzeug').setLevel(logging.ERROR)
logging.getLogger('root').setLevel(logging.CRITICAL)

# Iniciar o túnel manualmente
public_url = ngrok.connect(5000)
print(f"🚀 URL pública: {public_url}")

app = Flask(__name__)

modelo = joblib.load('/content/drive/MyDrive/data_bases/modelo_triagem.pkl')

@app.route('/prever', methods=['POST'])
def prever():
    dados = request.get_json()
    sintomas = input_to_list(dados)
    resultado = diagnostico(sintomas)
    return jsonify( resultado)

app.run()


🚀 URL pública: NgrokTunnel: "https://fb1293fcc508.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off
